In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
with open('/home/husein/translation/zsm_Latn.dev') as fopen:
    left = fopen.read().split('\n')
    
with open('/home/husein/translation/eng_Latn.dev') as fopen:
    right = fopen.read().split('\n')

In [3]:
left[0], right[0]

('Pada hari Isnin, Saintis daripada Sekolah Perubatan Universiti Stamford mengumumkan penemuan alat diagnostik baru yang boleh mengasingkan sel-sel mengikut jenis: cip kecil yang boleh dicetak yang boleh dihasilakn menggunakan pencetak standard inkjet untuk kira-kira satu sen A.S setiap satu.',
 'On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.')

In [4]:
len(left), len(right)

(998, 998)

In [5]:
import malaya

/home/husein/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [6]:
eflomal = malaya.alignment.ms_en.eflomal()

In [7]:
%%time

eflomal.align(left[:1], right[:1])

CPU times: user 7.78 ms, sys: 27.9 ms, total: 35.7 ms
Wall time: 324 ms


{'forward': [[(0, 0),
   (2, 1),
   (3, 2),
   (4, 3),
   (1, 4),
   (8, 5),
   (7, 6),
   (5, 7),
   (7, 8),
   (6, 9),
   (9, 10),
   (10, 11),
   (10, 12),
   (11, 14),
   (13, 15),
   (12, 16),
   (11, 17),
   (14, 18),
   (15, 19),
   (16, 20),
   (17, 21),
   (18, 22),
   (19, 23),
   (22, 24),
   (21, 25),
   (24, 26),
   (20, 27),
   (25, 28),
   (23, 29),
   (26, 30),
   (27, 31),
   (28, 32),
   (30, 33),
   (31, 34),
   (29, 35),
   (32, 36),
   (33, 38),
   (34, 39),
   (36, 40),
   (35, 41),
   (37, 42)]],
 'reverse': [[(0, 4),
   (1, 0),
   (2, 1),
   (3, 2),
   (4, 3),
   (5, 7),
   (6, 9),
   (7, 8),
   (9, 10),
   (10, 13),
   (11, 14),
   (12, 16),
   (13, 15),
   (14, 18),
   (15, 19),
   (16, 20),
   (17, 21),
   (18, 22),
   (19, 23),
   (20, 27),
   (21, 25),
   (22, 24),
   (23, 29),
   (24, 26),
   (25, 28),
   (26, 30),
   (27, 31),
   (28, 32),
   (29, 35),
   (30, 33),
   (31, 34),
   (32, 36),
   (33, 38),
   (34, 39),
   (35, 41),
   (36, 40),
   (37, 42),


In [8]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from collections import defaultdict
import re
import random

In [9]:
rev_rules_compound_normalizer = defaultdict(list)
for k, v in rules_compound_normalizer.items():
    rev_rules_compound_normalizer[v].append(k)

In [10]:
rev_rules_normalizer = defaultdict(list)
for k, v in rules_normalizer.items():
    rev_rules_normalizer[v].append(k)

In [11]:
rules_compound_normalizer_regex = (
    '(?:' + '|'.join(list(rev_rules_compound_normalizer.keys())) + ')'
)

In [12]:
# def _replace_compound(string, 
#                       rules_regex = rules_compound_normalizer_regex, 
#                       rules = rev_rules_compound_normalizer):
#     results = re.findall(rules_regex, string, flags=re.IGNORECASE
#     )
#     for r in results:
#         try:
#             string = string.replace(r, random.choice(rules[r.lower()]))
#         except:
#             pass
#     return string

def _replace_compound(string, rules = rev_rules_compound_normalizer):
    for k in list(rules.keys()):
        results = [(m.start(0), m.end(0)) for m in re.finditer(k, string, flags=re.IGNORECASE)]
        for r in results:
            sub = string[r[0]: r[1]]
            try:
                replaced = random.choice(rules[sub.lower()])
                if replaced:
                    if r[1] < len(string) and string[r[1]] != ' ':
                        continue
                    if r[0] - 1 > len(string) and string[r[0] - 1] != ' ':
                        continue

                    sub = case_of(sub)(replaced)
                    string = string[:r[0]] + sub + string[r[1]:]
            except:
                pass
    return string

In [13]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_


def replace_shortword(word):
    left, right, word_ = strip_punct(word)
    word_ = word_[len(left):]
    lower_word_ = word_.lower()
    if lower_word_ in rev_rules_normalizer:
        word_ = case_of(word_)(random.choice(rev_rules_normalizer[lower_word_]))
        word_ = f'{left}{word_}{right}'
        return word_
    else:
        return word
    
strip_punct('counters,')

('', ',', 'counters')

In [14]:
string = 'saya sangat cantik"'
[replace_shortword(word) for word in string.split()]

['gw', 'bngeet', 'cuantik"']

In [15]:
[replace_shortword(word) for word in left[1].split()]

['Penyelidik',
 'utama',
 'mengatakan',
 'bahawa',
 'ia',
 'mgkn',
 'menghasilkan',
 'pengesanan',
 'awl',
 'kanser,',
 'tuberkulosis,',
 'HIV',
 '&',
 'malaria',
 'kpada',
 'pesakit',
 'kat',
 'negara-negara',
 'berpendapatan',
 'low,',
 'kat',
 'manoi',
 'kadar',
 'kemandirian',
 'untk',
 'penyakit',
 'sperti',
 'kanser',
 'payu',
 'dara',
 'ialah',
 'separuh',
 'drpd',
 'd',
 'negara-negara',
 'yg',
 'lbih',
 'kya.']

In [16]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 2, max_replace = 7):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
            continue
        elif splitted_left[l].isupper() or splitted_right[r].isupper():
            continue
        elif splitted_left[l] == splitted_right[r]:
            continue
        elif splitted_right[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
            continue
        else:
            selected_alignment.append((l, r))
    
    count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
    
    selected = random.sample(selected_alignment, count_replace)
    for s in selected:
        splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])
    
    return ' '.join(splitted_left)

In [17]:
# random replace alignment
# random replace compound
# random replace word

In [18]:
random.random()

0.911006102501975

In [19]:
import copy

def augment(left, right, p_replace_alignment = 0.4, p_replace_shortword = 0.7):
    if random.random() > p_replace_alignment:
        alignment = eflomal.align([left], [right])['forward'][0]
        left = random_replace_alignment(left, right, alignment)
    
    left = _replace_compound(left, rules = copy.deepcopy(rev_rules_compound_normalizer))
    left = ' '.join([replace_shortword(word) if random.random() > p_replace_shortword else word for word in left.split()])
    return left

In [20]:
fast_text = malaya.language_detection.fasttext()

In [21]:
from tqdm import tqdm

new_left, new_right = [], []
for i in tqdm(range(len(left))):
    if len(left[i].split()) > 100 or len(right[i].split()) > 100:
        continue
    langs = fast_text.predict([left[i], right[i]])
    if langs[0] == 'eng':
        continue
    if langs[1] == 'malay':
        continue
    try:
        new_left_ = augment(left[i], right[i])
        if new_left_ != left[i]:
            new_left.append(new_left_)
            new_right.append(right[i])
    except Exception as e:
        pass

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 998/998 [02:01<00:00,  8.19it/s]


In [23]:
len(new_left)

953

In [24]:
len(new_right)

953

In [28]:
import json

with open('augmented-eng_Latn-zsm_Latn-noisy-ms-en-dev-set-v1.json', 'w') as fopen:
    json.dump({'en': new_right, 'ms': new_left}, fopen)